In [1]:
import pandas as pd
import csv
import numpy as np
import scipy as sc
import collections
import glob
import subprocess
import datetime

from matplotlib import pyplot as plt
from matplotlib import rc, rcParams
from matplotlib.patches import Rectangle

from itertools import islice
from scipy import constants as const

from scipy.interpolate import interp1d
from scipy.optimize import brentq

# ----------------------------------------------------------------
# cern specific modules for reading tfs files - outdated by pandas
# ----------------------------------------------------------------
import cern_pymad_domain_tfs as dom
import cern_pymad_io_tfs as io
# ****************************************************************

# ----------------------------------------------------------------
# constants
# ----------------------------------------------------------------
npart            = 1600
beampipediam     = 0.022    #[m]
lhcradius        = 2803.95  #[m]
dipolelength     = 14.3     #[m]
ionmass          = 193.72917484892244  #[GeV]
ioncharge        = 82.      #[el charge]
electronmassgev  = const.electron_mass * const.c**2/const.value('electron volt-joule relationship')/10**9 #[GeV]

# ----------------------------------------------------------------
# MADX - Twiss columns as dictionary
# ----------------------------------------------------------------
MADtwissColumns = {}

MADtwissColumns["RMatrixExtended"] = ["NAME", "KEYWORD", "PARENT", 
   "S", "L", "X", "PX", "Y", "PY", "T", "PT", "BETX", "BETY", "ALFX", 
   "ALFY", "MUX", "MUY", "DX", "DY", "DPX", "DPY", "HKICK", "VKICK", 
   "K0L", "K1L", "KMAX", "KMIN", "CALIB", "RE11", "RE12", 
   "RE13", "RE14", "RE15", "RE16", "RE21", "RE22", "RE23", "RE24", 
   "RE25", "RE26", "RE31", "RE32", "RE33", "RE34", "RE35", "RE36", 
   "RE41", "RE42", "RE43", "RE44", "RE45", "RE46", "RE51", "RE52", 
   "RE53", "RE54", "RE55", "RE56", "RE61", "RE62", "RE63", "RE64", 
   "RE65", "RE66"]

MADtwissColumns["LHCTwiss"] = ["NAME", "KEYWORD", "PARENT", "S", "L", 
   "LRAD", "KICK", "HKICK", "VKICK", "ANGLE", "K0L", "K1L", "K2L", 
   "K3L", "X", "Y", "PX", "PY", "BETX", "BETY", "ALFX", "ALFY", "MUX",
    "MUY", "DX", "DY", "DPX", "DPY", "KMAX", "KMIN", "CALIB", 
   "POLARITY", "APERTYPE", "N1", "TILT"]

MADtwissColumns["CTE"] = ["NAME","S","L","BETX","BETY","ALFX","ALFY","DX","DPX","DY","DPY","ANGL","K1L","K1S"]

# ----------------------------------------------------------------
# get column names of the tfs file
# ----------------------------------------------------------------
def get_tfsheader(tfsfile):
    headerdata =  pd.read_csv(tfsfile,delim_whitespace=True, skiprows=range(45),nrows=2,index_col=None)
    return headerdata.columns[1:].values

# ----------------------------------------------------------------
# get column names of the csv file
# ----------------------------------------------------------------
def get_csvheader(csvfile):
    headerdata =  pd.read_csv(csvfile,delim_whitespace=True,nrows=2,index_col=None)
    return headerdata.columns[0:].values

# ----------------------------------------------------------------
# MADX - Error template to include errors
# ----------------------------------------------------------------
errseqtemplate='''
USE, PERIOD=LHCB2;

EOPTION,ADD=TRUE,SEED=62971100;

SELECT, FLAG=ERROR, PATTERN="MQX*.*L5";
EALIGN, DX:=TGAUSS(1.5)*1.0E-4,DY:=TGAUSS(1.5)*1.0E-4,
        DS:=TGAUSS(1.5)*1.0E-4;
        
ERR = TGAUSS(1.0)*1.0E-4;

SELECT, FLAG=ERROR, PATTERN="MQX*.*L5";
EFCOMP, ORDER:=1, RADIUS=0.010,
    DKNR={0,ERR,ERR*1.,ERR*1.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0},
        DKSR={0,ERR,ERR*1.,ERR*1.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
        
'''

# ----------------------------------------------------------------
# MADX - SIXTRACK programs files path
# ----------------------------------------------------------------
sixpath          = '/afs/cern.ch/work/t/tomerten/sixtrack/'
madx             = '/afs/cern.ch/user/m/mad/bin/madx_dev'
madfilespath     = '/afs/cern.ch/work/t/tomerten/mad/'


# ----------------------------------------------------------------
# Off momentum ions delta 
#
# usage: 
# dpPb(dm,dq)
#
# dm = change in mass
# dq = change in charge
# ----------------------------------------------------------------
def dpPb(dm,dq):
    return (1+dm/ionmass)/(1+dq/ioncharge)-1

# ----------------------------------------------------------------
# Get the momentum (PC) from a tfs file 
#
# usage: 
# get_p(tfsfile)
#
# ----------------------------------------------------------------
def get_p(tfsfile):
    opt  = io.tfsDict(tfsfile)
    return opt[1]['pc']

# ----------------------------------------------------------------
# Get the kicker names in tfs file  
#
# usage: 
# get_kickernames(tfsfile,type)
# type = 'HKICKER' 
# ----------------------------------------------------------------
def get_kickernames(fn,ktype='HKICKER'):
    dat = pd.read_csv(fn,delim_whitespace=True,skiprows=range(45),index_col=None)
    dat = dat[dat.NAME != '%s']
    datheaders= dat.columns[1:]
    dat = pd.read_csv(filen,delim_whitespace=True,header=None,names=datheaders,skiprows=range(46),index_col=False)
    dat = dat[dat.KEYWORD != '%s']
    return dat[dat.KEYWORD == ktype].NAME.values

# ----------------------------------------------------------------
# MADX - R matrices for some optical elements used to calculate 
# impact locations
# ----------------------------------------------------------------

# dipole R matrix 3 X 3
def Mdipole(s,R):
    return np.array([[np.cos(s/R),R * np.sin(s/R),R * (1-np.cos(s/R))],\
                     [-1/R * np.sin(s/R),np.cos(s/R),np.sin(s/R)],\
                     [0,0,1]])

# dipole R matrix 6 X 6
def Mdipole6D(s,R):
    return np.array([
            [np.cos(s/R),R * np.sin(s/R),0,0,0,R * (1 - np.cos(s/R))],\
            [-1/R * np.sin(s/R),np.cos(s/R),0,0,0,np.sin(s/R)],\
            [0, 0, 1, s, 0, 0],\
            [0, 0, 0, 1, 0, 0],\
            [0, 0, 0, 0, 1, 0],\
            [0, 0, 0, 0, 0, 1]])

# dipole R matrix 6 X 6 for beam 4 note the sign change for the dispersion
def Mdipole6DB4(s,R):
    return np.array([
            [np.cos(s/R),R * np.sin(s/R),0,0,0,-R * (1 - np.cos(s/R))],\
            [-1/R * np.sin(s/R),np.cos(s/R),0,0,0,-np.sin(s/R)],\
            [0, 0, 1, s, 0, 0],\
            [0, 0, 0, 1, 0, 0],\
            [0, 0, 0, 0, 1, 0],\
            [0, 0, 0, 0, 0, 1]])

# focussing quadrupole R matrix 3 X 3
def Mquadf(s,k):
    return np.array([
            [np.cos(np.sqrt(np.absolute(k) * s)),\
                      (1/np.sqrt(k))*np.sin(np.sqrt(np.absolute(k) * s))],\
            [-np.sqrt(np.absolute(k))*np.sin(np.sqrt(np.absolute(k) * s)),\
                      np.cos(np.sqrt(np.absolute(k) * s))]])

# drift space R matrix 2 X 2
def Mdrift(s):
    return np.array([[1, s],[0,1]]) 

# drift space R matrix 6 X 6
def Mdrift6D(s):
    return np.array([[1, s, 0, 0, 0, 0],\
                    [0, 1, 0, 0, 0, 0],\
                    [0, 0, 1, s, 0, 0],\
                    [0, 0, 0, 1, 0, 0],\
                    [0, 0, 0, 0, 1, 0],\
                    [0, 0, 0, 0, 0, 1]])

# ----------------------------------------------------------------
# MADX - get initial parameters
#
# usage:
# get_initial(fn,deltap)
# fn = tfs file
# deltap = deltap to set for secondary beams
# ----------------------------------------------------------------
def get_initial(fn,deltap):
    dat = pd.read_csv(fn,delim_whitespace=True,header=None,names=MADtwissColumns["LHCTwiss"],skiprows=range(47),\
                      index_col=False)
    dcout = {}
    info = dat[dat.NAME == 'IP5']
    dcout['BETX'] = info.BETX.values[0]
    dcout['BETY'] = info.BETY.values[0]
    dcout['ALFX'] = info.ALFX.values[0]
    dcout['ALFY'] = info.ALFY.values[0]
    dcout['X'] = info.X.values[0]
    dcout['PX'] = info.PX.values[0]
    dcout['Y'] = info.Y.values[0]
    dcout['PY'] = info.PY.values[0]
    dcout['DELTAP'] = deltap
    return dcout

# ----------------------------------------------------------------
# MADX - adding madx code to save variables values
#
# usage:
# madxknob_save(knoblist,fn)
# fn = filename where to save the variables
# knoblist = list of variables to save  ex. ON_X2
# ----------------------------------------------------------------
def madxknob_save(knoblist,fn):
    outstr = '''
SELECT,FLAG=SAVE,CLEAR;
'''
    for name in knoblist:
        outstr += 'SELECT, FLAG=SAVE, CLASS=VARIABLE, PATTERN='+ name +';\n'
    outstr += 'SAVE,FILE={fnknob};\n'
    return outstr.format(fnknob=fn)

# ----------------------------------------------------------------
# MADX - matching of an orbit bump
# usage:
# madMatchBump(lhcseq,targetxc,targetel,correctorlist)
# lhcseq = sequence string 
# targetxc = desired displacement at target element
# targetel = element where the orbit displacement should match a certain value
# correctorlist = list of correctors to use for the construction of the bump
# ----------------------------------------------------------------
def madMatchBump(lhcseq,targetxc,targetel,correctorlist):
    linelist = []
    line = ''
    linelist.append('MATCH, SEQUENCE=' + lhcseq + line + ';')
    linelist.append('CONSTRAINT, SEQUENCE=' + lhcseq + ', RANGE=' + targetel + ', X=' + str(targetxc) + ';')
    linelist.append('CONSTRAINT, SEQUENCE=' + lhcseq + ', RANGE=' + correctorlist[-1] + ', X=0, PX=0;')
    for c in correctorlist:
        linelist.append('VARY, NAME='+c+'->HKICK, STEP=1.0E-6;')
    linelist.append('LMDIF, CALLS=500, TOLERANCE=1.0E-15;')
    linelist.append('ENDMATCH;')
    out=''
    for ln in linelist:
        out += ln + '\n' 
    return out

# ----------------------------------------------------------------
# MADX - generating madx beam commands to include in a madx input file
# usage:
# MADX_Beam(beam,seq='LHCB1',bv=1,energy=522340,charge=ioncharge,mass=ionmass,kbunch=518,npart=2.E8,\
#               ex=3.5e-6,ey=3.5e-6,et=0.000013665,sige=0.00015,sigt=0.09107)
# beam = 1 or 2 to give beam a name
# seq = sequence string 
# ----------------------------------------------------------------
def MADX_Beam(beam,seq='LHCB1',bv=1,energy=522340,charge=ioncharge,mass=ionmass,kbunch=518,npart=2.E8,\
              ex=3.5e-6,ey=3.5e-6,et=0.000013665,sige=0.00015,sigt=0.09107):
    gam = energy / mass
    ex  = ex /gam
    ey  = ey /gam
    beamdict={
        'beam'    : 'beam'+ str(beam),
        'seq'     : seq,
        'bv'      : bv,
        'energy'  : energy,
        'charge'  : charge,
        'mass'    : '{:18.14f}'.format(mass),
        'kbunch'  : kbunch,
        'npart'   : '{:.1E}'.format(npart),
        'ex'      : ex,
        'ey'      : ey,
        'et'      : et,
        'sige'    : sige,
        'sigt'    : sigt
    }
    return '''
{beam}:  BEAM, SEQUENCE={seq}, BV={bv}, ENERGY={energy},CHARGE={charge},
    MASS={mass}, KBUNCH={kbunch},NPART={npart},
    EX={ex}, EY={ey},ET={et},SIGE={sige},SIGT={sigt};
        
    '''.format(**beamdict)

# ----------------------------------------------------------------
# MADX - running twiss
#
# usage:
#  Twiss(seq,fn,targetxc=0.0,targetel='',correctorlist='',errorseq='',twisscols=MADtwissColumns["LHCTwiss"]
#                        ,beam=[MADX_Beam(1,seq='LHCB1',ey=1.5e-6),MADX_Beam(2,seq='LHCB2',ey=1.5e-6)])
# seq = sequence string 
# targetxc = desired displacement at target element
# targetel = element where the orbit displacement should match a certain value
# correctorlist = list of correctors to use for the construction of the bump
# errorseq = madx code for the errors to include in the twiss calculations
# twisscols = columns to write to the twiss output tfs file
# beam = list of two beam commands to use in the twiss
# ----------------------------------------------------------------
def Twiss(seq,fn,optstart='#S',optstop='#E',IPcycle='IP5',targetxc=0.0,targetel=' ',correctorlist=[],\
          errorseq='',twisscols=MADtwissColumns["LHCTwiss"],beam=[MADX_Beam(1,seq='LHCB1',ey=1.5e-6),
                                                                  MADX_Beam(2,seq='LHCB2',ey=1.5e-6)]):
    tw     = ''
    cycle  = '''
SEQEDIT, SEQUENCE={seq};
FLATTEN;
CYCLE, start={ip};
ENDEDIT;

    '''.format(ip=IPcycle,seq=seq)
    
    for i in twisscols:
        tw += i+','
    dformat ={
        'beam1'     : beam[0],
        'beam2'     : beam[1],
        'start'     : optstart,
        'stop'      : optstop,
        'cycle'     : cycle,
        'seq'       : seq,
        'fn'        : fn,
        'twcol'     : tw[:-1],
        'errors'    : errorseq 
    }
        
    if (targetel==' '):
        dformat['bumpmatch'] = ' '
    else:
        dformat['bumpmatch'] =madMatchBump(seq,targetxc,targetel,correctorlist)
        
    madin ='''
system,"ln -fns  /afs/cern.ch/eng/lhc/optics/runII/2015/ db5";
call, file="db5/lhcb4_as-built.seq";
call, file="db5/opt_inj_colltunes.madx";
call, file="db5/opt_800_800_800_3000_ion_coll.madx";

on_alice := 7000/6370.;
on_lcb   := 7000/6370.;
        
{beam1}
{beam2}
{cycle}

USE, PERIOD={seq};

SELECT,FLAG=TWISS,CLEAR;
        
SELECT,FLAG=TWISS,RANGE=#S/#E,COLUMN={twcol};
TWISS,SEQUENCE={seq},file={fn};

{bumpmatch}

{errors}

SELECT,FLAG=TWISS,CLEAR;
        
SELECT,FLAG=TWISS,RANGE=#S/#E,COLUMN={twcol};
TWISS,SEQUENCE={seq},file={fn};

'''.format(**dformat)
    fn = open('madin.madx','wt')
    fn.write(madin)
    fn.close()
    bashcommand = madx +' < ' + 'madin.madx' 
    subprocess.call(bashcommand,shell=True)
    return dformat['fn']

# ----------------------------------------------------------------
# MADX - transfermatrices
#
# usage:
# TransferMatrix(lhcseq,optstart,optstop,initdict,targetxc,targetel,correctorlist,errorseq='',fileext=''
#                     ,beam=[MADX_Beam(1,seq='LHCB1',ey=1.5e-6), MADX_Beam(2,seq='LHCB2',ey=1.5e-6)])
# lhcseq = sequence string 
# optstart = name of element where to start the twiss
# targetxc = desired displacement at target element
# targetel = element where the orbit displacement should match a certain value
# correctorlist = list of correctors to use for the construction of the bump
# errorseq = madx code for the errors to include in the twiss calculations
# twisscols = columns to write to the twiss output tfs file
# beam = list of two beam commands to use in the twiss
# ----------------------------------------------------------------
def TransferMatrix(lhcseq,optstart,optstop,initdict,targetxc,targetel,correctorlist,IPcycle='IP5',\
                   errorseq='',fileext='',beam=[MADX_Beam(1,seq='LHCB1',ey=1.5e-6),
                                                                  MADX_Beam(2,seq='LHCB2',ey=1.5e-6)]):
    fnstem = "BFPPbeamTransferMatrix" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    moufn  = fnstem + '\BFPPbeamTransferMatrix.mou'
    tw     = ''
    cycle  = '''
SEQEDIT, SEQUENCE={seq};
FLATTEN;
CYCLE, start={ip};
ENDEDIT;

    '''.format(ip=IPcycle,seq=lhcseq)
        
    for i in MADtwissColumns["RMatrixExtended"]:
        tw += i+','
    
    line = ''
    for k in initdict.keys():
        line += ','+ str(k)+ '=' + str(initdict[k])
        
    
    d = {\
        'beam1'        : beam[0],
        'beam2'        : beam[1],
        'seq'          : lhcseq,
        'start'        : optstart,
        'stop'         : optstop,
        'cycle'        : cycle,
        'twcol'        : tw[:-1],
        'transfertfsfn': 'bfppbeamtransfermatrix' + fileext 
             + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.tfs',
        'initdc'       : line,
        'bumpmatch'    : madMatchBump(lhcseq,targetxc,targetel,correctorlist),
        'fnknob'       : 'knob'
             + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.str',
        'errors'      : errorseq        
        }
    madin = '''
system,"ln -fns  /afs/cern.ch/eng/lhc/optics/runII/2015/ db5";
call, file="db5/lhcb4_as-built.seq";
call, file="db5/opt_inj_colltunes.madx";
call, file="db5/opt_800_800_800_3000_ion_coll.madx";

on_alice := 7000/6370.;
on_lcb   := 7000/6370.;
        
{beam1}
{beam2}
{cycle}

USE, PERIOD={seq};
        
SELECT,FLAG=TWISS,CLEAR;
        
SELECT,FLAG=TWISS,RANGE={start}/{stop},COLUMN={twcol};
TWISS,SEQUENCE={seq},file={transfertfsfn}{initdc}, RMATRIX=TRUE;

{bumpmatch}

{errors}

SELECT,FLAG=TWISS,CLEAR;
        
SELECT,FLAG=TWISS,RANGE={start}/{stop},COLUMN={twcol};
TWISS,SEQUENCE={seq},file={transfertfsfn}{initdc}, RMATRIX=TRUE;
    '''.format(**d)
    fn = open('madin.madx','wt')
    fn.write(madin)
    fn.close()
    bashcommand = madx +' < ' + 'madin.madx' 
    subprocess.call(bashcommand,shell=True)
    return d['transfertfsfn']

# ----------------------------------------------------------------
# MADX - Tracking of sigma matrix and generation of distributions 6D
#
# usage:
# TrackSigmaMatrix(infile,nameprevel,initdc,nparticles)
# infile = tfs input file
# nameprevel = name of element where to get the initial sigma matrix, at start of element
# initdc = initial conditions
# nparticles = number of particles in the distributions
# ----------------------------------------------------------------
def TrackSigmaMatrix(infile,nameprevel,initdc,nparticles):
    transfermatrixopt  = io.tfsDict(infile)
    prevelinex         = transfermatrixopt[0]["name"].index(nameprevel)-1
    gamma              = transfermatrixopt[1]['gamma']
    ex                 = transfermatrixopt[1]['ex']
    ey                 = transfermatrixopt[1]['ey']
    sige               = transfermatrixopt[1]['sige']
    sigt               = transfermatrixopt[1]['sigt']
    print transfermatrixopt[0]['name'][prevelinex-1]
    print transfermatrixopt[0]['name'][prevelinex]
    print transfermatrixopt[0]['name'][prevelinex+1]
    
    matrix = np.array([
        [transfermatrixopt[0]["re11"][prevelinex],transfermatrixopt[0]["re12"][prevelinex],
         transfermatrixopt[0]["re13"][prevelinex],transfermatrixopt[0]["re14"][prevelinex],
         transfermatrixopt[0]["re15"][prevelinex],transfermatrixopt[0]["re16"][prevelinex]],
        [transfermatrixopt[0]["re21"][prevelinex],transfermatrixopt[0]["re22"][prevelinex],
         transfermatrixopt[0]["re23"][prevelinex],transfermatrixopt[0]["re24"][prevelinex],
         transfermatrixopt[0]["re25"][prevelinex],transfermatrixopt[0]["re26"][prevelinex]],
        [transfermatrixopt[0]["re31"][prevelinex],transfermatrixopt[0]["re32"][prevelinex],
         transfermatrixopt[0]["re33"][prevelinex],transfermatrixopt[0]["re34"][prevelinex],
         transfermatrixopt[0]["re35"][prevelinex],transfermatrixopt[0]["re36"][prevelinex]],
        [transfermatrixopt[0]["re41"][prevelinex],transfermatrixopt[0]["re42"][prevelinex],
         transfermatrixopt[0]["re43"][prevelinex],transfermatrixopt[0]["re44"][prevelinex],
         transfermatrixopt[0]["re45"][prevelinex],transfermatrixopt[0]["re46"][prevelinex]],
        [transfermatrixopt[0]["re51"][prevelinex],transfermatrixopt[0]["re52"][prevelinex],
         transfermatrixopt[0]["re53"][prevelinex],transfermatrixopt[0]["re54"][prevelinex],
         transfermatrixopt[0]["re55"][prevelinex],transfermatrixopt[0]["re56"][prevelinex]],
        [transfermatrixopt[0]["re61"][prevelinex],transfermatrixopt[0]["re62"][prevelinex],
         transfermatrixopt[0]["re63"][prevelinex],transfermatrixopt[0]["re64"][prevelinex],
         transfermatrixopt[0]["re65"][prevelinex],transfermatrixopt[0]["re66"][prevelinex]]
    ])
    
    sigma0matrix = np.array([
        ex / 2. * np.array([initdc['BETX'],-initdc['ALFX'],0.,0.,0.,0.]),
        ex / 2. * np.array([-initdc['ALFX'],(2+initdc['ALFX']**2)/initdc['BETX'],0.,0.,0.,0.]),
        ey / 2. * np.array([0.,0.,initdc['BETY'],-initdc['ALFY'],0.,0.]),
        ey / 2. * np.array([0.,0.,-initdc['ALFY'],(2+initdc['ALFY']**2)/initdc['BETY'],0,0]),
        np.array([0.,0.,0.,0.,0.,0.]),
        np.array([0.,0.,0.,0.,0.,sige**2]),
        
    ])
    
    orbit = np.array([transfermatrixopt[0]["x"][prevelinex],
                     transfermatrixopt[0]["px"][prevelinex],
                     transfermatrixopt[0]["y"][prevelinex],
                     transfermatrixopt[0]["py"][prevelinex],
                     transfermatrixopt[0]["t"][prevelinex],
                     transfermatrixopt[0]["pt"][prevelinex]])
            
    sigma1matrix = np.dot(matrix,np.dot(sigma0matrix,np.transpose(matrix)))
    # print sigma0matrix,initdc['DELTAP'],np.sqrt(sigma1matrix[5,5])
    coordinates = np.transpose(
        np.array([
                np.random.normal(orbit[0],np.sqrt(sigma1matrix[0,0]),nparticles),
                np.random.normal(orbit[1],np.sqrt(sigma1matrix[1,1]),nparticles),
                np.random.normal(orbit[2],np.sqrt(sigma1matrix[2,2]),nparticles),
                np.random.normal(orbit[3],np.sqrt(sigma1matrix[3,3]),nparticles),
                np.random.normal(0,sigt,nparticles),
                np.random.normal(initdc['DELTAP'],np.sqrt(sigma1matrix[5,5]),nparticles)#initdc['DELTAP']
                
                         ])
    )
    return coordinates

# ----------------------------------------------------------------
# MADX - calculating impact coordinates
#
# usage:
# impactcoordinates6D(coord,ax,r,s0,ldipole,delta,mass,deltam,p0,filen,beam4=False
# coord = output of tracksigmatrix, input distributions to track
# ax = diameter of beam pipe assumed circular
# r = bending radius of main lhc dipoles
# s0 = s value of start of element where to track
# delta = output of dpPb function
# mass = particle mass
# deltam = mass change of the particles
# p0 = reference momentum
# filename = filename where to save the impact distributions
# beam4 = if tracking with beam 4 in madx set to true
# ----------------------------------------------------------------
def impactcoordinates6D(coord,ax,r,s0,ldipole,delta,mass,deltam,p0,filen,beam4=False):
    def f(s,axs,rr,y,vect):
        if beam4:
            out = -np.sqrt(axs**2-y**2)-np.dot(Mdipole6DB4(s,rr),vect)[0]
        else:
            out = -np.sqrt(axs**2-y**2)-np.dot(Mdipole6D(s,rr),vect)[0]
        return out

    
    def fd(s,axs,rr,y,vect):
        if beam4:
            out = -np.sqrt(axs**2-y**2)-(np.dot(Mdrift6D(s),np.dot(Mdipole6DB4(ldipole,rr),vect)))[0]
        else:
            out = -np.sqrt(axs**2-y**2)-(np.dot(Mdrift6D(s),np.dot(Mdipole6D(ldipole,rr),vect)))[0]
        return out
    
    spos = []
    for i in range(len(coord)):
        try:
            zero = brentq(f,0,100,args=(ax,r,coord[i,2],coord[i]))
            if zero > ldipole:
#                 plt.plot(range(0,100,1),[fd(j,ax,r,coord[i,2],coord[i]) for j in range(0,100,1)])
                spos.append(ldipole + brentq(fd,0,100,args=(ax,r,coord[i,2],coord[i])))
            else:
                spos.append(zero)
        except:
            continue
    #plt.show()
   
    # matrix6d = np.array([np.dot(Mdipole6D(spos[i],r),coord[i]) for i in range(len(coord))])
    if beam4:
        matrix6d = np.array([np.dot(Mdrift6D(spos[i]-ldipole),np.dot(Mdipole6DB4(ldipole,r),coord[i]))
            if spos[i] > ldipole else np.dot(Mdipole6DB4(spos[i],r),coord[i]) 
            for i in range(len(spos))])
    else:
        matrix6d = np.array([np.dot(Mdrift6D(spos[i]-ldipole),np.dot(Mdipole6D(ldipole,r),coord[i]))
                if spos[i] > ldipole else np.dot(Mdipole6D(spos[i],r),coord[i]) 
                for i in range(len(spos))])
#     print spos
    scol  = pd.Series(data=spos)+s0
    xcol  = pd.Series(data=matrix6d[:,0])
    pxcol = pd.Series(data=matrix6d[:,1])
    ycol  = pd.Series(data=matrix6d[:,2])
    pycol = pd.Series(data=matrix6d[:,3])
    Ecol  = pd.Series(data=
                      np.sqrt((mass + deltam)**2 + (p0 * (1 + matrix6d[:,5] - delta) * (1 + deltam/mass))**2))             
    
    outdf             = pd.DataFrame(scol,columns=['s[m]'])
    outdf['x[mm]']    = pd.Series(data=xcol)*1000
    outdf['px[1e-3]'] = pd.Series(data=pxcol)*1000
    outdf['y[mm]']    = pd.Series(data=ycol)*1000
    outdf['py[1e-3]'] = pd.Series(data=pycol)*1000
    outdf['E[GeV]']   = pd.Series(data=Ecol) 
    outdf.to_csv(filen + '.csv',index=False)
    return outdf

In [2]:
%save -f madxmodule.py 1

The following commands were written to file `madxmodule.py`:
import pandas as pd
import csv
import numpy as np
import scipy as sc
import collections
import glob
import subprocess
import datetime

from matplotlib import pyplot as plt
from matplotlib import rc, rcParams
from matplotlib.patches import Rectangle

from itertools import islice
from scipy import constants as const

from scipy.interpolate import interp1d
from scipy.optimize import brentq

# ----------------------------------------------------------------
# cern specific modules for reading tfs files - outdated by pandas
# ----------------------------------------------------------------
import cern_pymad_domain_tfs as dom
import cern_pymad_io_tfs as io
# ****************************************************************

# ----------------------------------------------------------------
# constants
# ----------------------------------------------------------------
npart            = 1600
beampipediam     = 0.022    #[m]
lhcradius

In [62]:
print madxknob_save(['ON_A2'],'knob.sav')


SELECT,FLAG=SAVE,CLEAR;

SELECT, FLAG=SAVE, CLASS=VARIABLE, PATTERN=ON_A2;
SAVE,FILE=knob.sav;



In [49]:
Twiss('lhcb2','test.tfs',IPcycle='IP5',targetxc=0.0,targetel='',correctorlist='',errorseq='',twisscols=MADtwissColumns["LHCTwiss"])

'test.tfs'

In [54]:
s0=403.8427102
impactfilelist=['impactIP5left_bp22_noerr',
 'impactIP5left_bp22_mqxalignfield',
 'impactIP5left_bp22_mqmlfield',
 'impactIP5left_bp22_mqmlalign',
 'impactIP5left_bp22_mqmlalignfield',
 'impactIP5left_bp22_mqfield',
 'impactIP5left_bp22_mqalign',
 'impactIP5left_bp22_mqml5field',
 'impactIP5left_bp22_mqml5align']
impactcoordinates6D(TrackSigmaMatrix(transfermatrixfilelist[0],
                                                               'MB.B11L5.B2',initialdictlist[0],npart)
                    ,beampipediam,lhcradius,s0,dipolelength,
                                   dpPb(0,-1),ionmass,
                                   -electronmassgev,
                                   get_p(transfermatrixfilelist[0]),impactfilelist[0],beam4=True)

MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2
[7.872651589107038, 8.195560332106245, 9.368528154793609, 8.562150764905791, 9.106927301031769, 8.219005321262992, 7.426593658880267, 8.122616110325191, 8.520477446204014, 7.719350080445459, 9.343561637872034, 9.943977607591068, 8.537256093127064, 9.208157928509783, 8.488544043044621, 8.406706040077745, 10.09272323963738, 8.598123939594156, 8.847261398111158, 9.711048751060702, 7.529760368567157, 8.723715311794463, 8.706301176636714, 7.501174877177081, 8.617250279149888, 8.633388431186697, 8.70469084025204, 7.941206295947483, 8.660389676901836, 8.226952863370451, 7.2539609440568915, 8.247932036142883, 9.289905146916013, 7.362331381741368, 8.496933916366553, 7.38660593342567, 9.251034057391509, 8.664197283830276, 9.504695174919457, 7.848831606881671, 8.806097534908714, 9.105036980990024, 8.846042411235455, 7.403687219717469, 7.921400384117328, 8.966313016466525, 7.412309554961281, 8.214291836423671, 7.882384667206634, 8.974908546213102, 7.29490813322818

,s[m],x[mm],px[1e-3],y[mm],py[1e-3],E[GeV]
0,411.715362,-21.998361,-0.433523,-0.290428,-0.002781,522281.694285
1,412.038271,-21.987703,-0.446296,0.719983,-0.001888,522334.085899
2,413.211238,-21.999707,-0.425617,0.093312,-0.002160,522380.431712
3,412.404861,-21.999832,-0.430931,-0.150001,-0.007485,522425.610295
4,412.949638,-21.976563,-0.452303,0.996801,-0.002023,522248.859750
5,412.061716,-21.998722,-0.427012,0.205681,-0.003827,522374.106949
6,411.269304,-21.994433,-0.420587,-0.452981,0.005645,522374.355391
7,411.965326,-21.998507,-0.436671,-0.186353,0.008607,522326.796337
8,412.363188,-21.999807,-0.430525,-0.067250,0.002932,522255.183834
9,411.562060,-21.999461,-0.461025,-0.138467,0.002015,522238.104717


In [2]:
# setting bump amplitude
bumpamp = 0.00
# set list of files for base twiss
#     basefilelist=[
#         'lhcb2-twiss-noerrb5p0.tfs',
#         'lhcb2-twiss-mqxalignfieldb5p0.tfs',
#         'lhcb2-twiss-mqmlfieldb5p0.tfs',
#         'lhcb2-twiss-mqmlalignb5p0.tfs',
#         'lhcb2-twiss-mqmlalignfieldb5p0.tfs',
#         'lhcb2-twiss-mqfieldb5p0.tfs',
#         'lhcb2-twiss-mqalignb5p0.tfs',
#         'lhcb2-twiss-mqml5fieldb5p0.tfs',
#         'lhcb2-twiss-mqml5alignb5p0.tfs'        
#     ]
basefilelist=[
    'lhcb2-twiss-test.tfs'        
]
# correctorlist
corrlist = np.array(['MCBCH.7L5.B2', 'MCBCH.9L5.B2', 'MCBH.13L5.B2'])

# twiss file list
basetwissfilelist = [Twiss('lhcb2',basefilelist[i],targetxc=bumpamp,targetel="MQ.11L5.B2",\
                           correctorlist=corrlist,errorseq='',twisscols=MADtwissColumns["LHCTwiss"])\
                    for i in range(len(basefilelist))]

In [3]:
print basetwissfilelist
!ls lhcb2*test.tfs


['lhcb2-twiss-test.tfs']
lhcb2-twiss-test.tfs


In [4]:
# initial parameters for BFPP
initialdictlist = [get_initial(fn,dpPb(0,-1)) for fn in basetwissfilelist]
    
#     fileextlist = [
#         'noerrb5p0',
#         'mqxalignfieldb5p0',
#         'mqmlfieldb5p0',
#         'mqmlalignb5p0',
#         'mqmlalignfieldb5p0',
#         'mqfieldb5p0',
#         'mqalignb5p0',
#         'mqml5fieldb5p0',
#         'mqml5alignb5p0'        
#     ]

In [5]:
fileextlist = [
    'test'       
]

transfermatrixfilelist = [
    TransferMatrix('LHCB2','IP5','S.DS.L5.B2',initialdictlist[i],bumpamp,"MQ.11L5.B2",corrlist,\
                   errorseq='',\
                   fileext=fileextlist[i]) for i in range(len(basetwissfilelist))]

# writing transfermatrixfilelist to file for importing and plotting
dftransfermatrixfilelist       = pd.DataFrame(pd.Series(data=transfermatrixfilelist),columns=['filename'])
dftransfermatrixfilelist.to_csv('transfermatrixfilenamelist' + '.csv',index=False)
    

In [13]:
madxtracks[0][madxtracks[0].NAME == 'DRIFT_54']['PX']

142   -0.000035
146   -0.000097
160   -0.000280
164   -0.000342
180    0.000201
184    0.000139
198   -0.000336
202   -0.000399
224    0.000458
236    0.000334
250   -0.000124
258   -0.000186
262   -0.000249
Name: PX, dtype: float64

In [33]:
coordtest[:,5]

array([ 0.01232511,  0.01256301,  0.01236563, ...,  0.01244013,
        0.01275438,  0.01244671])

In [38]:
def Mdipole(s,R):
    return np.array([[np.cos(s/R),\
                      R * np.sin(s/R),\
                    - R * (1 - np.cos(s/R))],
                     [-1/R * np.sin(s/R),\
                      np.cos(s/R),
                     - np.sin(s/R)],\
                     [0,0,1]])
print np.mean(coordtest[:,0])
print np.mean(coordtest[:,1])
x = [np.dot(Mdipole(14.3,lhcradius),[coordtest[j,0],coordtest[j,1],coordtest[j,-1]])[0]
     for j in range(len(coordtest)) ]
px = [np.dot(Mdipole(14.3,lhcradius),[coordtest[j,0],coordtest[j,1],coordtest[j,-1]])[1]
     for j in range(len(coordtest)) ]
print np.mean(x)
print np.mean(px)

-0.0184463833506
-0.000398762940756
-0.0245985749314
-0.000461681569197


In [26]:
print madxtracks[0][madxtracks[0].NAME == 'DRIFT_54']['X'].loc[202]
print madxtracks[0][madxtracks[0].NAME == 'DRIFT_54']['PX'].loc[202]
print madxtracks[0][madxtracks[0].NAME == 'MB.B11L5.B2']['X']
print madxtracks[0][madxtracks[0].NAME == 'MB.B11L5.B2']['PX']

-0.018447381
-0.0003986215899
203   -0.024592
Name: X, dtype: float64
203   -0.000461
Name: PX, dtype: float64


In [77]:
print np.mean(coordtest[:,5])
print np.mean(coordtest3[:,5])
print np.mean(coordtest3[:,1])
print initialdictlist
print lhcradius * (1-np.cos(14.3/lhcradius))*coordtest[:,5]


0.0123442067123
3.2292813603e-07
-0.000398330066884
[{'DELTAP': 0.012345679012345734, 'PY': -4.9851662220000004e-14, 'BETY': 0.79999988980000003, 'BETX': 0.80000027780000005, 'PX': -0.00014500018480000001, 'Y': -1.9267131209999999e-13, 'X': -2.0599804069999998e-10, 'ALFY': -6.7097718320000001e-08, 'ALFX': -1.3766434630000003e-06}]
[ 0.00044034  0.00045355  0.00045236 ...,  0.00045182  0.00044584
  0.00044988]


In [11]:
coordtest = TrackSigmaMatrix(transfermatrixfilelist[0],'MB.B11L5.B2',initialdictlist[0],npart)

MCS.B11L5.B2
DRIFT_54
MB.B11L5.B2


In [125]:
plt.hist(coordtest[:,0],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest[:,1],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest2[:,0],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest2[:,1],weights=[1/1600. for i in range(1600)])
plt.show()

In [126]:
plt.hist(coordtest[:,2],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest[:,3],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest2[:,2],weights=[1/1600. for i in range(1600)])
plt.hist(coordtest2[:,3],weights=[1/1600. for i in range(1600)])
plt.show()

In [7]:
fnlist = pd.read_csv('transfermatrixfilenamelist.csv',delim_whitespace=True,index_col=None)['filename'].values
headerdata = get_tfsheader(fnlist[0])
print headerdata
madxtracks = [pd.read_csv(fn,delim_whitespace=True, skiprows=range(47),names=headerdata,index_col=None)\
              for fn in fnlist]

['NAME' 'KEYWORD' 'PARENT' 'S' 'L' 'X' 'PX' 'Y' 'PY' 'T' 'PT' 'BETX' 'BETY'
 'ALFX' 'ALFY' 'MUX' 'MUY' 'DX' 'DY' 'DPX' 'DPY' 'HKICK' 'VKICK' 'K0L'
 'K1L' 'KMAX' 'KMIN' 'CALIB' 'RE11' 'RE12' 'RE13' 'RE14' 'RE15' 'RE16'
 'RE21' 'RE22' 'RE23' 'RE24' 'RE25' 'RE26' 'RE31' 'RE32' 'RE33' 'RE34'
 'RE35' 'RE36' 'RE41' 'RE42' 'RE43' 'RE44' 'RE45' 'RE46' 'RE51' 'RE52'
 'RE53' 'RE54' 'RE55' 'RE56' 'RE61' 'RE62' 'RE63' 'RE64' 'RE65' 'RE66']


In [10]:
# R * (1-np.cos(s/R))
def Mdipole(s,R):
    return np.array([[np.cos(s/R),\
                      R * np.sin(s/R),\
                     R * (1 - np.cos(s/R))],
                     [-1/R * np.sin(s/R),\
                      np.cos(s/R),
                      np.sin(s/R)],\
                     [0,0,1]])
stest = np.linspace(1,14.3,100)

rcParams['figure.figsize']=18,25

x = [[np.dot(Mdipole(stest[i],lhcradius),[coordtest[j,0],coordtest[j,1],coordtest[j,-1]])[0]\
     for i in range(len(stest))] for j in range(len(coordtest)) ]

y = [[np.dot(Mdipole(stest[i],lhcradius),[coordtest2[j,0],coordtest2[j,1],coordtest2[j,-1]])[0]\
     for i in range(len(stest))] for j in range(len(coordtest2)) ]

z = [[np.dot(Mdipole(stest[i],lhcradius),[coordtest3[j,0],coordtest3[j,1],coordtest3[j,-1]])[0]\
     for i in range(len(stest))] for j in range(len(coordtest2)) ]

ax1 =  plt.subplot(311)
for j in range(5):
    plt.plot(stest+403.84,x[j])
    plt.plot(stest+403.84,z[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

ax2 =  plt.subplot(312,sharex=ax1)
for j in range(5):
    plt.plot(stest+403.84,y[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

ax3 =  plt.subplot(313,sharex=ax1)
for j in range(5):
    plt.plot(stest+403.84,z[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

plt.show()

NameError: name 'coordtest' is not defined

In [70]:
def Mdipole6D(s,R):
    return np.array([
            [np.cos(s/R),R * np.sin(s/R),0,0,0,R * (1 - np.cos(s/R))],\
            [-1/R * np.sin(s/R),np.cos(s/R),0,0,0,np.sin(s/R)],\
            [0, 0, 1, s, 0, 0],\
            [0, 0, 0, 1, 0, 0],\
            [0, 0, 0, 0, 1, 0],\
            [0, 0, 0, 0, 0, 1]])

stest = np.linspace(1,14.3,100)

rcParams['figure.figsize']=18,25

x = [[np.dot(Mdipole6D(stest[i],lhcradius),coordtest[j])[0]\
     for i in range(len(stest))] for j in range(len(coordtest)) ]

y = [[np.dot(Mdipole6D(stest[i],lhcradius),coordtest2[j])[0]\
     for i in range(len(stest))] for j in range(len(coordtest2)) ]

z = [[np.dot(Mdipole6D(stest[i],lhcradius),coordtest3[j])[0]\
     for i in range(len(stest))] for j in range(len(coordtest2)) ]

ax1 =  plt.subplot(311)
for j in range(5):
    plt.plot(stest+403.84,x[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

ax2 =  plt.subplot(312,sharex=ax1)
for j in range(5):
    plt.plot(stest+403.84,y[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

ax3 =  plt.subplot(313,sharex=ax1)
for j in range(5):
    plt.plot(stest+403.84,z[j])
plt.plot(madxtracks[0]['S'],madxtracks[0]['X'])  
plt.xlim(402,420)
plt.ylim(-0.030,-0.015)

plt.show()
# plt.savefig('MADXtroubles.png')

In [168]:
htmlstring='''<!DOCTYPE html>
<html>
<head><h1>MADX Troubles</h1> </head>
<body>

<div>
    <h2>BFPP Beams MADX and tracking with R matrices of distributions </h2>
    <center>Longest line is the madx orbit from tfs file, the shorter lines are single 
    particles tracked using 6D dipole. In the first plot I used the function as defined by Michaela for 
    2011. In the second plot initial parameters at end of MB were used. Finally in the last plot
    all coordinates at beginning of MB where used except for px and py where the values at the end of the MB 
    where used. Notice that in the first plot the angles (px) don't agree with the madx tfs track.</center>
    <img src="MADXtroubles.png" width="1100" height="1500">
</div>
</body>
</html>'''
f = open('Madxtroubles.html','w')
f.write(htmlstring)
f.close()